# Crawl API

In [1]:
import requests
from calendar import monthrange
import datetime as dt
import time
import csv
import os

* Sử dụng api của https://api.weather.com
* Dữ liệu được lấy trong 10 năm từ 2010 đến 2020.
* Dữ liệu được lấy từ trạm thời tiết của sân bay Tân Sơn Nhất(VVTS). 

In [2]:
if not os.path.isdir('data'):
    os.mkdir('data')

cur = dt.datetime(2010, 1, 1)
end = dt.datetime(2020, 1, 1)

location = 'VVTS:9:VN'

* Sử dụng public api key lấy từ https://www.wunderground.com thông qua crawl html.

In [3]:
# Lấy api key
def get_api_key():
    r = requests.get('https://www.wunderground.com/')
    text = r.text
    text = text[text.find('apiKey=') + len('apiKey='):]
    text = text[:text.find('&')]
    return text


def save(path, data):
    with open(path, 'w', newline='') as csvfile:
        fieldnames = list(data[0].keys())
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerows(data)


# Tính ngày cuối cùng của tháng
def end_date(date):
    day = monthrange(month=date.month, year=date.year)[1]
    return date.replace(day=day)


# Thử lại nếu bị timeout
def retry(url):
    r = requests.get(url)
    
    while r.status_code != 200:
        time.sleep(1)
        r = requests.get(url)
        print('timeout')
        
    return r


# Lấy dữ liệu từ api
def get_data(startDate, endDate):
    url = f'https://api.weather.com/v1/location/'\
            f'{location}/observations/historical.json?units=m&'\
            f'apiKey={api_key}&startDate={startDate}&endDate={endDate}'
        
    r = retry(url)
    data = r.json()
    
    if 'observations' in data:
        return data['observations']
    
    return []

In [4]:
start = time.time()
api_key = get_api_key()
print(f'API key: {api_key}')

data = []
# Lấy từng tháng
while cur != end:
    startDate = cur.strftime("%Y%m%d")
    endDate = end_date(cur).strftime("%Y%m%d")


    data.extend(get_data(startDate, endDate))
    time.sleep(0.5)

    if cur.month < 12:
        cur = dt.datetime(cur.year, cur.month + 1, 1)
    else:
        cur = dt.datetime(cur.year + 1, 1, 1)

print(f'Total: {len(data)}')
print(f'Elapsed Time: {time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - start))}')
save(f'data/data.csv', data)

API key: 6532d6454b8aa370768e63d6ba5a832e
Total: 171694
Elapsed Time: 00h 01m 55s
